# 시계열 데이터 시각화

시계열 데이터를 시각화를 연습합니다. 시계열 데이터 중 좋은 예로 사용할 수 있는 데이터는 주식데이터일 것 같습니다.
`FinanceDataReader`라이브러리를 이용하여 `Kospi` 주가 정보를 읽어오고 이를 이용하여 월별 수익률을 계산합니다.

시계열 데이터의 경우 일정 기간을 기준으로 통계정보를 추출하고 이를 지표로 사용하는 경우가 많습니다.
KOSPI 주가지수를 분석하는 예제로 시계열 데이터를 시각화합니다.


In [ ]:
#| echo: false
import pandas as pd
import numpy as np
import FinanceDataReader as fdr


def monthly_returns(name, start_date, ):
    # start=str
    df = fdr.DataReader(name, start = str(start_date))
    monthly_df = df.resample('M').ffill()

    monthly_df['Adj Close(+1)'] = monthly_df['Adj Close'].shift(1)
    monthly_df['monthly returns(%)'] = (monthly_df['Adj Close'] / monthly_df['Adj Close(+1)'] - 1)*100
    monthly_df['monthly returns(6개월평균)'] = monthly_df['monthly returns(%)'].rolling(6, min_periods=1).mean()

    return monthly_df


# stock_name = 'US500'
stock_name = 'KS11'
monthly_df= monthly_returns(stock_name, (pd.Timestamp.now().year - 3))
monthly_df

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# fig = go.Figure()
fig = make_subplots(rows=3, cols=1)

df = monthly_df.iloc[-24:, :]
fig.add_hline(y = 1.117, line_color='red')
color_info =['lightgray']*12 + ['gray']*12 + ['blue']*12

fig.add_trace(go.Bar(x=df.index, y=df['monthly returns(%)'],name='monthly returns(%)'),
              row=1,
              col=1)

fig.add_trace(go.Scatter(x=df.index, y=df['monthly returns(6개월평균)'], name= "monthly return(avg. 6M)"),
              row=2,
              col=1)
fig.add_hline(y = 0, line_color='black', row=2, col=1, opacity=0.3)
fig.add_trace(go.Scatter(x=df.index, y=df['Adj Close'], name='Kospi'),
              row=3,
              col=1)

config = {
    'displayModeBar': False,
}

fig.update_layout(
    legend = dict(
        orientation="h", # 가로 방향으로
        yanchor="top", y=1.1, # y축 방향 위치 설정
        xanchor="left", x=0.01, # x축 방향 위치 설정
    ),
    xaxis = dict(
        dtick = 'M1',
        tickangle = 45,
        tickfont = dict(size = 9),
    ),
    height = 700,
    width = 700,
)

fig.show(config=config)

Pandas 데이터 프레임으로 KOSPI 주가지수 정보를 로드하고 각각의 정보를 3개의 sub plot으로 구성하여 같이 비교할 수 있도록 위와 같이 구성합니다.
예제를 통해서 subplot을 생성하는 방법에 대해서 이해할 수 있었고 특정 시간 단위로 시계열 데이터를 처리하는 방법도 배울 수 있었습니다.